In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import os
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import multiprocessing

In [2]:
html_parser = {
    'title': 'art-title',
    'department': 'art-product-name',
    'author': 'art-author',
    'date': 'art-datetime',
    'text': 'art_paragraph'
    }

def get_articles(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    articles = soup.find_all(class_='result-item-link',href=True)
    return articles

def get_article_data(link, driver):
    data = {
        'title': None,
        'department': None,
        'author': None,
        'date': None,
        'text': None,
        'link': link
    }
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for key, value in html_parser.items():
        try:
            if key == 'text':
                paragraphs = soup.find_all(class_=value)
                data[key] = " ".join([paragraph.text for  paragraph in paragraphs]).replace('\n',"").strip()
            else:
                data[key] = soup.find(class_=value).text.replace('\n',"").strip()
        except:
            print(f"Key: {key} not found in {link}")
    return data

def scrap_page(url):
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=selenium")
    chrome_options.add_argument('headless')
    driver = webdriver.Chrome(options=chrome_options)
    articles = get_articles(url)
    data = []
    for article in articles:
        link='https://classic.wyborcza.pl/archiwumGW/'+article['href']
        data.append(get_article_data(link, driver))
    driver.close()
    return data

def valdidate_csv(csv_file):
    if not os.path.exists(csv_file):
        print(f"Creating file {csv_file}")
        column_names = ['Title', 'Department', 'Author', 'Date', 'Text', 'Link']
        with open(csv_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(column_names)
    else:
        print(f"Appending to {csv_file}")

In [3]:
csv_file = 'wyborcza.csv'
valdidate_csv('wyborcza.csv')

Appending to wyborcza.csv


In [4]:
def process_page(page):
    link_page = f'https://classic.wyborcza.pl/archiwumGW/0,160510.html?searchForm=&datePeriod=0&initDate=2015-01-01&endDate=2023-01-01&publicationsString=1%3B5&author=&page={page}&sort=OLDEST'
    data_list = scrap_page(link_page)
    print(f"Page {page}")
    return data_list

with multiprocessing.Pool(4) as pool:
    results = pool.map(process_page, range(10))

with open(csv_file, 'a', newline='') as file:
    writer = csv.writer(file)
    for data_list in results:
        for data in data_list:
            writer.writerow([data['title'], data['department'], data['author'], data['date'], data['text'], data['link']])

Key: author not found in https://classic.wyborcza.pl/archiwumGW/7971829/KOMIKS-JACKA-FRASIA
Key: author not found in https://classic.wyborcza.pl/archiwumGW/7971837/2014---NASZE-NAJLEPSZE-ZDJECIA
Key: author not found in https://classic.wyborcza.pl/archiwumGW/7971834/2015--O-NICH-BEDZIE-GLOSNO
Page 2
Page 0
Page 4
Page 1
Page 7
Page 8
Page 5
Page 9
Page 6


WebDriverException: Message: unknown error: cannot parse internal JSON template: Line: 1, column: 1, Unexpected token.
Stacktrace:
#0 0x55f119309f83 <unknown>
#1 0x55f118fc2cf7 <unknown>
#2 0x55f118ffddba <unknown>
#3 0x55f118ffe2fa <unknown>
#4 0x55f118ff8d2c <unknown>
#5 0x55f118ff726e <unknown>
#6 0x55f11904780c <unknown>
#7 0x55f11903be53 <unknown>
#8 0x55f119003dd4 <unknown>
#9 0x55f1190051de <unknown>
#10 0x55f1192ce531 <unknown>
#11 0x55f1192d2455 <unknown>
#12 0x55f1192baf55 <unknown>
#13 0x55f1192d30ef <unknown>
#14 0x55f11929e99f <unknown>
#15 0x55f1192f7008 <unknown>
#16 0x55f1192f71d7 <unknown>
#17 0x55f119309124 <unknown>
#18 0x7f3ccb7b2947 start_thread
#19 0x7f3ccb838860 __clone3
